# Fetch Rewards Coding Exercise - Data Analyst

## First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

Diagram sent as separate file, using mySQL to create the Diagram.

Review data as below.

In [1]:
import pandas as pd
import numpy as np
import json
import re
from pandas import json_normalize

In [2]:
brands = pd.read_json('brands.json',lines = True)
users = pd.read_json('users.json',lines = True)
receipts = pd.read_json('receipts.json',lines = True)

## Brands Data

In [3]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [4]:
nan_cols = [i for i in brands.columns if brands[i].isnull().any()] # check which columns have null values
nan_cols

['category', 'categoryCode', 'topBrand', 'brandCode']

In [5]:
brands['_id'] = brands['_id'].apply(lambda x: x.get('$oid')) # get the values from $oid

In [6]:
cpg = pd.DataFrame([x for x in brands['cpg']]) # there's a nested dic from cpg, change it to a dataframe
cpg.head(2)

,$id,$ref
0,{'$oid': '601ac114be37ce2ead437550'},Cogs
1,{'$oid': '5332f5fbe4b03c9a25efd0ba'},Cogs


In [7]:
cpg.rename(columns={'$id':'cpg.id', '$ref':'cpg.ref'}, inplace=True)

In [8]:
cpg['cpg.id'] = cpg['cpg.id'].apply(lambda x: x.get('$oid')) # get the values from cpg's $oid

In [9]:
cpg.head(1)

,cpg.id,cpg.ref
0,601ac114be37ce2ead437550,Cogs


In [10]:
brands_clean = pd.merge(brands, cpg, left_index=True, right_index=True) # create new df, merged and deleted original column
brands_clean.drop(columns='cpg', inplace=True)
brands_clean.head(3)

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg.id,cpg.ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs


In [11]:
brands_clean._id.nunique() == len(brands) 

True

In [12]:
any(brands_clean.duplicated())

False

## Users Data

In [13]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [14]:
nan_cols = [i for i in users.columns if users[i].isnull().any()]
nan_cols

['lastLogin', 'signUpSource', 'state']

In [15]:
users['_id'] = users['_id'].apply(lambda x: x.get('$oid')) # get the values from $oid, $date
users['createdDate'] = users['createdDate'].apply(lambda x: x.get('$date') if str(x) != str(np.nan) else x)
users['lastLogin'] = users['lastLogin'].apply(lambda x: x.get('$date') if str(x) != str(np.nan) else x)

In [16]:
users['createdDate'] = pd.to_datetime(users['createdDate']//1000, unit='s') # convert to datetime
users['lastLogin'] = pd.to_datetime(users['lastLogin']//1000, unit='s')

In [17]:
users.head(2)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


In [18]:
len(users.duplicated())

495

In [19]:
any(users.duplicated())

True

In [20]:
users.groupby(['_id','createdDate','lastLogin'],as_index=False)['role'].count().rename(columns={'role':'counts'})\
.sort_values(ascending=False, by = 'counts')

,_id,createdDate,lastLogin,counts
0,54943462e4b07e684157a532,2014-12-19 14:21:22,2021-03-05 16:52:23,20
9,5fc961c3b8cfca11a077dd33,2020-12-03 22:08:03,2021-02-26 22:39:16,20
7,5fa41775898c7a11a6bcef3e,2020-11-05 15:17:09,2021-03-04 16:02:02,18
32,5ff5d15aeb7c7d12096d91a2,2021-01-06 15:03:54,2021-01-06 15:08:10,18
3,59c124bae4b0299e55b0f330,2017-09-19 14:07:54,2021-02-08 16:42:58,18
...,...,...,...,...
64,5ffca30604929111f6e92525,2021-01-11 19:12:06,2021-01-11 19:12:06,1
63,5ffc9d9f04929111f6e92456,2021-01-11 18:49:03,2021-01-11 18:49:03,1
61,5ffc9001b3348b11c93388b6,2021-01-11 17:50:57,2021-01-11 17:50:57,1
58,5ff8da7eb3348b11c9337b72,2021-01-08 22:19:42,2021-01-08 22:19:42,1


In [21]:
users_clean = users.drop_duplicates(keep='first') # drop duplicates but keep the first occurance

In [22]:
len(users_clean)

212

In [23]:
users_clean['_id'].nunique() == len(users_clean)

True

In [24]:
users_clean.head(3)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 15:25:28,2021-01-03 15:25:28,consumer,Email,WI


## Receipt Data

In [25]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [26]:
receipts.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent',
       'userId'],
      dtype='object')

In [27]:
nan_cols = [i for i in receipts.columns if receipts[i].isnull().any()]
nan_cols

['bonusPointsEarned',
 'bonusPointsEarnedReason',
 'finishedDate',
 'pointsAwardedDate',
 'pointsEarned',
 'purchaseDate',
 'purchasedItemCount',
 'rewardsReceiptItemList',
 'totalSpent']

In [28]:
receipts.bonusPointsEarnedReason.unique()

array(['Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'All-receipts receipt bonus',
       'Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 3 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 6 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 4 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       nan, 'COMPLETE_PARTNER_RECEIPT', 'COMPLETE_NONPARTNER_RECEIPT'],
      dtype=object)

In [29]:
receipts['_id'] = receipts['_id'].apply(lambda x: x.get('$oid')) 

In [30]:
### Cleaning all the dates from Receipts using similar approach as above and loop
for i in receipts.columns:
    if re.findall(r'date|Date', i):
        receipts[i] = receipts[i].apply(lambda x: x.get('$date') if str(x) != str(np.nan) else x)
        receipts[i] = pd.to_datetime(receipts[i]//1000, unit='s')
receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


In [31]:
(receipts['createDate'] != receipts['dateScanned']).sum() # check if createDate same as dateScanned

0

In [32]:
### Cleaning "rewardsReceiptItemList" from Receipts
receipts = receipts.explode('rewardsReceiptItemList') # make the list to be dictionary-like instead of a list

In [33]:
receipts['rewardsReceiptItemList']

0       {'barcode': '4011', 'description': 'ITEM NOT F...
1       {'barcode': '4011', 'description': 'ITEM NOT F...
1       {'barcode': '028400642255', 'description': 'DO...
2       {'needsFetchReview': False, 'partnerItemId': '...
3       {'barcode': '4011', 'description': 'ITEM NOT F...
                              ...                        
1115                                                  NaN
1116                                                  NaN
1117    {'barcode': 'B076FJ92M4', 'description': 'muel...
1117    {'barcode': 'B07BRRLSVC', 'description': 'thin...
1118                                                  NaN
Name: rewardsReceiptItemList, Length: 7381, dtype: object

In [34]:
receiptItemList = json_normalize(receipts['rewardsReceiptItemList'])

In [35]:
receiptItemList.head(3)

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
receiptItemList.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

In [37]:
receipts_clean = pd.merge(receipts, receiptItemList, left_index=True, right_index=True, how='outer')
receipts_clean.drop(columns='rewardsReceiptItemList', inplace=True)
receipts_clean.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned_x,purchaseDate,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned_y,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
len(receipts_clean.duplicated())

13643

In [39]:
any(receipts_clean.duplicated())

True

In [40]:
receipts_clean.drop_duplicates(keep='first',inplace=True)

In [41]:
len(receipts_clean)

6524

In [42]:
## I cleaned and merged some tables as one for analysis at the end, 
## but we could save them as separated tables as well for future usage.

#receipts_clean.to_csv('./receipts_clean.csv', index=False) 
#users_clean.to_csv('./users_clean.csv', index=False)
#brands_clean.to_csv('./brands_clean.csv', index=False)

## Second: Write a query that directly answers a predetermined question from a business stakeholder

In [43]:
# Run SQL queries in Python Pandas Dataframe
!pip install pandasql


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import pandasql as ps

### 1. What are the top 5 brands by receipts scanned for most recent month?

In [46]:
query = """WITH cte AS
           (SELECT \
           brandCode, \
           strftime('%Y%m', datetime(dateScanned)) AS scanned_year_month,
           COUNT(_id) AS number,
           DENSE_RANK() OVER (ORDER BY strftime('%Y%m', datetime(dateScanned)) DESC, COUNT(_id) DESC) as rank\
           FROM receipts_clean\
           WHERE brandCode IS NOT null AND strftime('%Y%m', datetime(dateScanned)) = (SELECT MAX(strftime('%Y%m', datetime(dateScanned))) FROM receipts_clean)
           GROUP BY 1, 2)\
           
           SELECT\
           brandCode,\
           scanned_year_month AS most_recent_month,\
           number\
           FROM cte\
           WHERE rank <= 5;\
        
           """
         
ps.sqldf(query)

,brandCode,most_recent_month,number
0,HY-VEE,202103,11
1,KASHI,202103,2
2,DOLE,202103,2
3,BUSH'S BEST,202103,2
4,NATURE'S PATH ORGANIC,202103,1
5,LIPTON,202103,1
6,LAURA'S LEAN BEEF,202103,1
7,KRAFT,202103,1
8,JUST BARE,202103,1
9,HILLSHIRE FARM,202103,1


For the most recent month, (not sure this is just the beginning of the month or end of the month.)
* we can see that some brands have the same receipt numbers
* HY-VEE outnumbers other brands


### 2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [47]:
## top 5 brands | receipts scanned | comparing previous and most recent month
## assuming previous month = previous month's records from current month

## group by year_month and brand to get each brand's numbers in that month
## use dense_rank in order there's tie in the ranks

query = """WITH cte AS
           (SELECT \
           brandCode, \
           strftime('%Y%m', datetime(dateScanned)) AS scanned_year_month,
           COUNT(*) AS number,
           DENSE_RANK() OVER (ORDER BY strftime('%Y%m', datetime(dateScanned)) DESC, COUNT(_id) DESC) as rank\
           FROM receipts_clean\
           WHERE brandCode IS NOT null AND strftime('%Y%m', datetime(dateScanned)) < (SELECT MAX(strftime('%Y%m', datetime(dateScanned))) FROM receipts_clean)
           GROUP BY 1, 2)\
           
           SELECT\
           brandCode,\
           scanned_year_month AS previous_month,\
           number\
           FROM cte\
           WHERE rank <= 5 AND scanned_year_month = (SELECT MAX(scanned_year_month) FROM cte);\
        
           """
         
ps.sqldf(query)

,brandCode,previous_month,number
0,HY-VEE,202102,72
1,BEN AND JERRYS,202102,15
2,LIGHT & FIT GREEK,202102,9
3,DOLE,202102,9
4,NATURE'S PATH ORGANIC,202102,7
5,LAURA'S LEAN BEEF,202102,7
6,KLEENEX,202102,7
7,JUST BARE,202102,7
8,HILLSHIRE FARM,202102,7
9,LIGHT & FIT,202102,6


From the numbers, we should assume that records from the most recent months is not a completed month. And comparing the most recent month and previous month:
* "HY-VEE" still outnumbers other brands. 

* However, one thing we should notice is that "BEN AND JERRYS" & "LIGHT & FIT GREEK" were on top of the list from previous month, we don't see them on the most recent month. 

We could look into these 2 brands to see what are the reasons causing the changes.

## Third: Evaluate Data Quality Issues in the Data Provided

### 1. Duplicated Data Issue

In [48]:
users.groupby(['_id','createdDate','lastLogin'],as_index=False)['role'].count().rename(columns={'role':'counts'})\
.sort_values(ascending=False, by = 'counts')

,_id,createdDate,lastLogin,counts
0,54943462e4b07e684157a532,2014-12-19 14:21:22,2021-03-05 16:52:23,20
9,5fc961c3b8cfca11a077dd33,2020-12-03 22:08:03,2021-02-26 22:39:16,20
7,5fa41775898c7a11a6bcef3e,2020-11-05 15:17:09,2021-03-04 16:02:02,18
32,5ff5d15aeb7c7d12096d91a2,2021-01-06 15:03:54,2021-01-06 15:08:10,18
3,59c124bae4b0299e55b0f330,2017-09-19 14:07:54,2021-02-08 16:42:58,18
...,...,...,...,...
64,5ffca30604929111f6e92525,2021-01-11 19:12:06,2021-01-11 19:12:06,1
63,5ffc9d9f04929111f6e92456,2021-01-11 18:49:03,2021-01-11 18:49:03,1
61,5ffc9001b3348b11c93388b6,2021-01-11 17:50:57,2021-01-11 17:50:57,1
58,5ff8da7eb3348b11c9337b72,2021-01-08 22:19:42,2021-01-08 22:19:42,1


From previous data cleansing, we found out that there's multiple duplicated users login data. Using id, createdDate and lastLogin as groups, we can see what users have duplicated records and how many records are there. Same goes with receipts data

In [49]:
receipts.groupby(['_id','dateScanned','finishedDate','modifyDate'],as_index=False)['pointsAwardedDate'].count().rename(columns={'pointsAwardedDate':'counts'})\
.sort_values(ascending=False, by = 'counts').head()

,_id,dateScanned,finishedDate,modifyDate,counts
370,600f2fc80a720f0535000030,2021-01-25 20:53:28,2021-01-25 22:04:23,2021-01-28 22:37:02,459
371,600f39c30a7214ada2000030,2021-01-25 21:36:03,2021-01-26 03:06:03,2021-01-26 03:07:40,450
344,600a1a8d0a7214ada2000008,2021-01-22 00:21:32,2021-01-22 00:30:13,2021-01-22 00:31:04,203
259,60049d9d0a720f05f3000094,2021-01-17 20:27:09,2021-01-17 20:40:36,2021-01-17 20:41:17,194
254,60025cb80a720f05f300008d,2021-01-16 03:25:44,2021-01-16 03:31:55,2021-01-16 03:32:05,185


The duplicated data will cause misleading analysis. We should understand what causes the duplicated records, since we already have user id or receipt id, maybe we can check with engineer team to see if there is system or webiste probelms, or what devices cause this problem the most, trying to get a more accurate data in the future.

### 2. Missing Data Issue

In [50]:
query = """WITH cte AS
           (SELECT \
           brandCode, \
           strftime('%Y%m', datetime(dateScanned)) AS scanned_year_month,
           COUNT(*) AS number,
           DENSE_RANK() OVER (ORDER BY strftime('%Y%m', datetime(dateScanned)) DESC, COUNT(_id) DESC) as rank\
           FROM receipts_clean\
           WHERE strftime('%Y%m', datetime(dateScanned)) < (SELECT MAX(strftime('%Y%m', datetime(dateScanned))) FROM receipts_clean)
           GROUP BY 1, 2)\
           
           SELECT\
           brandCode,\
           scanned_year_month AS previous_month,\
           number\
           FROM cte\
           WHERE rank <= 5 AND scanned_year_month = (SELECT MAX(scanned_year_month) FROM cte);\
        
           """
         
ps.sqldf(query)

,brandCode,previous_month,number
0,None,202102,177
1,HY-VEE,202102,72
2,BEN AND JERRYS,202102,15
3,LIGHT & FIT GREEK,202102,9
4,DOLE,202102,9
5,NATURE'S PATH ORGANIC,202102,7
6,LAURA'S LEAN BEEF,202102,7
7,KLEENEX,202102,7
8,JUST BARE,202102,7
9,HILLSHIRE FARM,202102,7


For the analysis here, miss values are ignored. 
But in the future, we could discuss with team to see if those data should be filled at first or drop some unnecessary columns, or how to deal with those missing values would be better.
Taking brandCode as example, we actually have lots of missing values from the brandCode, if these brands are filled with real brands, then it might affect the ranking of the brands.

## Fourth: Communicate with Stakeholders

Hi team,

This is Charlene from the Data team.

After reviewing the users, brands, and receipts data, here's some issues I'd like to clarify and discuss with you:

1. The data we have right now are not easy for analysis and visulization usage. I think we could automate this process of transforming the data to tables so that it will be more efficient in the future to perform further analysis.  

2. We have duplicated users records in the users data:
   I noticed that some of the users have serveral records at the exactly same date and time, such as user_id:54943462e4b07e684157a532, it showed 20 times for this user at the same login time and date. I could set up a meeting with engineering team first to see if this is caused by the system.

3. Missing values from the data, especially in receipts data:
   I noticed that there're some data not collected or recorded in our dataset, such as brandCode in the receipts data. If we want to know which brand is doing better comparing othes, it would be better to collect all of these data down so that the brand ranking would be much correct. Now I just use the data we have to do the analysis and ignore those missing values, but I'd love to hear your advice of how to deal with them.


I'll like to go through the details and would also love to hear your ideas during our weekly meeting. Please let me know if you are not able to join, I'd also upload meeting notes afterwards.



Best,

Charlene